# DistilBERT - Training on TweetEval (Ran on Colab)

## Install requirements and import packages

In [1]:
!pip install pandas numpy scikit-learn torch transformers datasets
## If running locally - uncomment and run the below package install requirements.
"""
!pip install "transformers[torch]"
!pip install "accelerate>=0.26.0"
"""


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


'\n!pip install "transformers[torch]"\n!pip install "accelerate>=0.26.0"\n'

In [2]:
import numpy as np
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, multilabel_confusion_matrix

/Users/sanshritbakshi/PycharmProjects/CS6120_final/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load the Dataset

In [3]:
dataset = load_dataset("dair-ai/emotion")
label_names = dataset["train"].features["label"].names
num_labels  = len(label_names)
print(label_names)

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']


## Tokenizer and Data Preprocessing

In [6]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess(examples):
    """
    Tokenize a batch of text examples and attach their labels for PyTorch training.

    Args:
        examples (dict):
            A batch from a HuggingFace Dataset containing:
              - "text": List[str], raw input strings to be classified.
              - "label": List[int], integer class labels corresponding to each text.

    Returns:
        dict:
            A dict with the following keys:
              - "input_ids": List[List[int]], token IDs for each input, padded/truncated to max_length.
              - "attention_mask": List[List[int]], mask (1 for real tokens, 0 for padding).
              - "labels": List[int], the original integer labels.
    """
    tokens = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    tokens["labels"] = examples["label"]
    return tokens

tokenized = dataset.map(preprocess, batched=True)

tokenized.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)

train_ds = tokenized["train"]
val_ds   = tokenized["validation"]
test_ds  = tokenized["test"]

Map: 100%|██████████| 2000/2000 [00:00<00:00, 47689.64 examples/s]


## Training the model

In [ ]:
def compute_metrics(eval_pred):
    """
    Compute evaluation metrics for multi-label classification.

    Args:
        eval_pred (tuple):
            A tuple of (logits, labels)
            - logits: np.ndarray of shape (batch_size, num_labels)
              Raw outputs from the model’s classification head.
            - labels: np.ndarray of shape (batch_size, num_labels)
              Ground-truth multi-hot vectors (0/1).
        threshold (float, optional):
            Probability cutoff for deciding positive labels after sigmoid.
            Defaults to 0.3.

    Returns:
        dict:
            {
                "f1_micro": float,
                    The micro-averaged F1 score across all labels.
                "subset_accuracy": float,
                    The fraction of samples where the predicted multi-hot
                    vector exactly matches the ground truth.
            }
    """
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro")
    }

# Load model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels
)

#TrainingArguments
training_args = TrainingArguments(
    output_dir="./bert_emotion",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=9,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    logging_steps=50,
    report_to="none",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics= compute_metrics,
    tokenizer=tokenizer
)

# Train
trainer.train()

# Evaluate
print("Validation Metrics:", trainer.evaluate())
print("Test Metrics      :", trainer.predict(test_ds).metrics)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/xk/v43z00vn3mj3lb_2ryh_5wxm0000gn/T/ipykernel_21564/2832962973.py:56: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.202100,0.188821,0.930500,0.904665


/Users/sanshritbakshi/PycharmProjects/CS6120_final/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Validation Metrics: {'eval_loss': 0.18882125616073608, 'eval_accuracy': 0.9305, 'eval_f1_macro': 0.9046647877134344, 'eval_runtime': 8.244, 'eval_samples_per_second': 242.601, 'eval_steps_per_second': 7.642, 'epoch': 1.0}


/Users/sanshritbakshi/PycharmProjects/CS6120_final/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Test Metrics      : {'test_loss': 0.19353729486465454, 'test_accuracy': 0.924, 'test_f1_macro': 0.8860765428569849, 'test_runtime': 8.3926, 'test_samples_per_second': 238.304, 'test_steps_per_second': 7.507}


# Results

In [9]:
from sklearn.metrics import confusion_matrix
import pandas as pd

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
ds = load_dataset("dair-ai/emotion", split="test")
y_true, y_pred = [], []
for ex in ds:
    text = ex["text"]
    true = ex["label"]
    enc  = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    ).to(device)

    with torch.no_grad():
        logits = model(**enc).logits

    # single‐label prediction via softmax
    probs = torch.softmax(logits, dim=-1)[0].cpu().numpy()
    pred  = int(np.argmax(probs))

    y_true.append(true)
    y_pred.append(pred)

y_true = np.array(y_true)
y_pred = np.array(y_pred)

#Overall accuracy
acc = accuracy_score(y_true, y_pred)
print(f"Overall Accuracy: {acc:.4f}\n")

# 5) Per‐class precision / recall / F1
print("Classification Report:")
print(classification_report(
    y_true,
    y_pred,
    target_names=label_names,
    zero_division=0
))

#Confusion matrix
cm = confusion_matrix(y_true, y_pred)

#Unfold into TP, FP, FN, TN per class
rows = []
total = cm.sum()
for idx, label in enumerate(label_names):
    TP = cm[idx, idx]
    FP = cm[:, idx].sum() - TP
    FN = cm[idx, :].sum() - TP
    TN = total - (TP + FP + FN)
    rows.append({
        "Class": label,
        "TN": TN,
        "FP": FP,
        "FN": FN,
        "TP": TP,
    })

#Display
df_conf_stats = pd.DataFrame(rows).set_index("Class")
display(df_conf_stats)

Overall Accuracy: 0.9240

Classification Report:
              precision    recall  f1-score   support

     sadness       0.96      0.96      0.96       581
         joy       0.96      0.93      0.94       695
        love       0.78      0.88      0.83       159
       anger       0.92      0.93      0.93       275
        fear       0.91      0.88      0.90       224
    surprise       0.76      0.77      0.77        66

    accuracy                           0.92      2000
   macro avg       0.88      0.89      0.89      2000
weighted avg       0.93      0.92      0.92      2000



,TN,FP,FN,TP
Class,,,,
sadness,1394,25,25,556
joy,1275,30,48,647
love,1801,40,19,140
anger,1704,21,19,256
fear,1756,20,26,198
surprise,1918,16,15,51


## Testing on Text

In [11]:
model.eval()

def predict_emotion_local(text: str):
    """
    Predict the most likely emotion for a given piece of text.

    This function tokenizes the input string, passes it through the
    fine-tuned BERT model, applies a softmax to obtain class probabilities,
    and returns the label with the highest probability along with its confidence score.

    Args:
        text (str):
            A single input string to classify.

    Returns:
        tuple[str, float]:
            - emotion: The name of the predicted emotion label (from `label_names`).
            - confidence: The softmax probability of the predicted label, in [0.0, 1.0].
    """
    enc = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    ).to(device)

    with torch.no_grad():
        logits = model(**enc).logits          # shape (1, num_labels)
        probs  = torch.softmax(logits, dim=-1)[0].cpu().numpy()

    best_id    = int(probs.argmax())
    emotion    = label_names[best_id]
    confidence = probs[best_id]

    return emotion, confidence

for txt in [
    "I just got my dream job—feeling on top of the world!",
    "Why does everything always go wrong for me?",
    "That plot twist in the movie made me jump!",
    "I can not wait to go home"
]:
    emo, conf = predict_emotion_local(txt)
    print(f"> {txt}\n→ {emo} ({conf:.1%})\n")

> I just got my dream job—feeling on top of the world!
→ joy (98.0%)

> Why does everything always go wrong for me?
→ anger (55.8%)

> That plot twist in the movie made me jump!
→ joy (35.3%)

> I can not wait to go home
→ sadness (42.6%)

